# **Convex Hull** Visualizer
Nama : Patrick Amadeus Irawan \
NIM : 13520109 \
Kelas : K-01


----
### **Table of Content**
1. Library Implementation + Dependency
    - QuickSort implementation
    - Helper Function implementation
    - Convex Hull algorithm implementation
2. Dataset Pick
3. Columns Pick
4. Points QuickSort Implementation
5. Hull Points Gathering
6. Visualization

----
###### 1. Library Implementation + Dependency
Base Library and Dependency

In [ ]:
# base library
import pandas as pd  
import matplotlib.pyplot as plt  

# datasets property
from sklearn import datasets

#### **QuickSort** Implementation for points
###### Sorting all points **(X,Y)** increasing by **X** then **Y**

In [ ]:
def quick_sort(points):
    if len(points) <= 1:
        return points
    else:
        pivot = points[0]
        less = [point for point in points[1:] if (point[0] < pivot[0]) or (point[0] == pivot[0] and point[1] < pivot[1])]
        greater = [point for point in points[1:] if (point[0] > pivot[0]) or (point[0] == pivot[0] and point[1] >= pivot[1])]
        return quick_sort(less) + [pivot] + quick_sort(greater)

#### **Helper** Function Implementation

In [ ]:
# Point to Line distance finder
def dist_pt_line(p1,p2,pt):
    '''
    Gather linear coefficient of line Ax + By + C = 0 and
    using the distance formula to find the distance between point and line
    
    params : p1, p2, pt = tuples of (x,y)
    return : float
    '''
    A,B,C = (p2[1] - p1[1]) , (p1[0] - p2[0]) , (p1[0]*(p1[1] - p2[1]) + p1[1]*(p2[0] - p1[0]))
    return abs(A*pt[0] + B*pt[1] + C)/((A**2 + B**2)**(0.5))

# linear value finder
def linear_value(p1,p2,pt):
    '''
    Gather linear coefficient of line Ax + By + C = 0 and
    input X and Y from pt to find the linear value

    params : p1, p2, pt = tuples of (x,y)
    return : float

    Example : 
    5x + 2y + 3
    pt = (2,3)

    5*2 + 2*3 + 3 = 15
    '''
    A,B,C = (p2[1] - p1[1]) , (p1[0] - p2[0]) , (p1[0]*(p1[1] - p2[1]) + p1[1]*(p2[0] - p1[0]))
    return A*pt[0] + B*pt[1] + C

#### Divide and Conquer **Convex Hull** algorithm Implementation

In [ ]:
def myConvexHull(points, p1 = None, p2 = None , types = 0):
    '''
    Gather points and find the convex hull of the points (set of points)
    points = initial set of points
    p1,p2 = mininimum and maximum point of the set of points (based on quick sort)
    types = 0 : initial state
            1 : upper region
            2 : lower region
    base -> 0 <= len(points) < 2
    recursion -> len(points) >= 2

    params : points = list of tuples of (x,y)
    return : list of tuples of (x,y)
    '''

    # ---- Base condition ---- #
    if len(points) == 0:
        return []
    if len(points) == 1:
        return [points[0]]
    
    # ---- Type 0 : Initial 2 area separation ---- #
    if not types:
        # find the minimum and maximum point
        p_min , p_max = points[0] , points[-1] 
        
        upper,lower = [],[]
        for point in points:
            if linear_value(p_min,p_max,point) < 0:
                upper.append(point)
            elif linear_value(p_min,p_max,point) > 0:
                lower.append(point)
        return [p_min] + myConvexHull(upper,p_min,p_max,1) + [p_max] + myConvexHull(lower,p_min,p_max,-1)

    # ---- Type 1 & 2 : Upper and Lower region ---- #

    # Find the point with the maximum distance to the line
    distance, p_max = 0, None
    for point in points:
        if dist_pt_line(p1,p2,point) > distance:
            distance = dist_pt_line(p1,p2,point)
            p_max = point
    
    # Find the points in the upper and lower region
        # left : region between p1 and p_max
        # right : region between p_max and p2
    left,right = [],[]
    for point in points:
        if linear_value(p1,p_max,point) * types < 0:
            left.append(point)
        if linear_value(p_max,p2,point) * types < 0:
            right.append(point)

    # Recursion order separation
    if types == 1:
        return myConvexHull(left , p1 , p_max, types) + [p_max] + myConvexHull(right , p_max , p2, types)
    else:
        return myConvexHull(right , p_max , p2, types) + [p_max] + myConvexHull(left , p1 , p_max, types)

----
###### 2. Dataset Pick <a id = "two"></a>
### Choose any classification dataset from **Toy Datasets** properties as listed below
1. Iris Plants Dataset
2. Wine recognition dataset
3. Breast cancer wisconsin (diagnostic) dataset

*Enter the number of wanted dataset within input*

In [ ]:
data_sets = [
    (datasets.load_iris(), "Iris Plants Dataset"),
    (datasets.load_wine(), "Wine Recognition Dataset"),
    (datasets.load_breast_cancer(), "Breast Cancer Wisconsin (Diagnostic) Dataset"),
]

data_sets_choice = int(input("Enter the dataset number : "))

while data_sets_choice > len(data_sets) or data_sets_choice < 1:
    data_sets_choice = int(input("Please enter the valid dataset number : "))

data = data_sets[data_sets_choice - 1][0]

Chosen Dataset **DataFrame** Preview

In [ ]:
df = pd.DataFrame(data.data, columns=data.feature_names)
df['Target'] = pd.DataFrame(data.target)
df.head()

Enumerate Target Label from **Dataset**

In [ ]:
n_target = len(df.Target.value_counts())
d_target = {}
for i in range(n_target):
    d_target[i] = (df.Target.value_counts().index[i])

-----
###### 3. Columns Pick
### Select 2 fields to be projected with **Convex Hull**
###### (*Please choose different columns*)

In [ ]:
idx = 0
msg = ""
for i in range(len(df.columns)):
    msg += (str(i+1)+". "+str(df.columns[i]) + "\n")
print("%s dataset have %d columns which enumerated as below : " % (data_sets[data_sets_choice - 1][1],len(df.columns)))
print(msg)

In [ ]:
idX,idY = int(input("Pick column 1 to plot : ")),int(input("Pick column 2 to plot : "))
while idX <= 0 or idX > len(df.columns) or idY < 0 or idY > len(df.columns):
    idX,idY = int(input("Pick column 1 to plot : ")),int(input("Pick column 2 to plot : "))
while idX == idY:
    idY = int(input("(Please Pick the different column)Pick column 2 to plot : "))
x,y = df.columns[idX-1],df.columns[idY-1]

----
###### 4. Points QuickSort Implementation

In [ ]:
# gather and sort points for chosen columns
d_points = {}
for i in d_target:
    d_points[i] = quick_sort([(df[x].values[j] , df[y].values[j]) for j in range(len(df[x])) if df.Target.values[j] == i])
                            # -------------- ^^^^^ explanation ^^^^^ ----------------- #
# explanation : for every target value (i), gather all points and sort them by absis then ordinate

-----
###### 5. Hull Points Gathering

In [ ]:
hull_points = {}

for i in range(n_target):
    hull_points[i] = myConvexHull(d_points[i])


------
###### 6. Visualization
### Final Scatter Plot with **Convex Hull** 

In [ ]:
color_pallete = ["blue","green","red","purple","magenta","yellow","black","white"] # for plotting

f = plt.figure()
f.set_figwidth(7.5)
f.set_figheight(5)

for i in d_points:
    x, y = [pts[0] for pts in d_points[i]],[pts[1] for pts in d_points[i]]
    plt.scatter(x,y,c = color_pallete[i], label = data.target_names[i] ,s=20)

for i in hull_points:
    size = len(hull_points[i])
    for j in range(len(hull_points[i])):
        plt.plot([hull_points[i][j % size][0], hull_points[i][(j + 1)% size][0]], [hull_points[i][j % size][1], hull_points[i][(j + 1)% size][1]] , c = color_pallete[i], linewidth = 0.75)

plt.title("Convex Hull of %s" % data_sets[data_sets_choice - 1][1])
plt.xlabel(df.columns[int(idX) - 1])
plt.ylabel(df.columns[int(idY) - 1])
plt.legend()

plt.show()